# Pitchfork review analysis
### made by $DS^3$
June 26, 2019

In [9]:
# for database
import psycopg2

# for data science
import numpy as np
import pandas as pd

# functions
from dale_functions_mod_3 import import_data

# for statistics
from scipy import stats

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Import data

In [4]:
genres_df = import_data("genres", 'genres_df')
reviews_df = import_data('reviews', 'reviews_df')
labels_df = import_data('labels', 'labels_df')

Merge and pair down columns

In [5]:
clean_reviews_df = reviews_df[['reviewid', 'score']]
reviews_labels_df = clean_reviews_df.merge(labels_df, how='left', on='reviewid')
music_df = reviews_labels_df.merge(genres_df, how='left', on='reviewid')

In [6]:
music_df[music_df['genre'] == 'folk/country'].score.count()
country_scores = music_df[music_df['genre'] == 'folk/country'].score
electronic_scores = music_df[music_df['genre'] == 'electronic'].score

Functions for stats

In [7]:
def welch_t(a, b):
    
    """ Calculate Welch's t statistic for two samples. """
    numerator = a.mean() - b.mean()
    
    # “ddof = Delta Degrees of Freedom”: the divisor used in the calculation is N - ddof, 
    #  where N represents the number of elements. By default ddof is zero.
    
    denominator = np.sqrt(a.var(ddof=1)/a.size + b.var(ddof=1)/b.size)
    
    return np.abs(numerator/denominator)

def welch_df(a, b):
    
    s1 = a.var(ddof=1)
    s2 = b.var(ddof=1)
    n1 = a.size
    n2 = b.size
    numerator = (s1/n1 + s2/n2)**2
    denominator = (s1/ n1)**2/(n1 - 1) + (s2/ n2)**2/(n2 - 1)
    
    return numerator/denominator

In [10]:
t = welch_t(country_scores, electronic_scores)
de_free = welch_df(country_scores, electronic_scores)
p = 1 - stats.t.cdf(t, de_free)
p

1.6913470624047022e-11

The genre's were rated very differently between folk/country and electronic with a welch's P value of 1.69e-11

In [ ]:
touch_and_go_df = music_df[music_df['label'] =='touch and go']
other_label_df = music_df[music_df['label'] !='touch and go']

touch_and_go_scores = music_df[music_df['label'] =='touch and go'].score
other_scores = music_df[music_df['label'] !='touch and go'].score

#alpha = .05
t = welch_t(touch_and_go_scores, other_scores)
de_free = welch_df(touch_and_go_scores, other_scores)
p = 1 - stats.t.cdf(t, de_free)
p

We refuted the null hypothisis that the the label 'touch and go' was rated differently the p value = .26 for welch's test much higher than .05 alpha we chose beforehand